In [ ]:
# !pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl'
!pip install --no-deps "../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
!pip install --no-deps "../input/resnest/resnest-0.0.5-py3-none-any.whl"

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data.sampler import SequentialSampler

# import ensemble_boxes
from  torchvision.models.utils import load_state_dict_from_url
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.backbone_utils import BackboneWithFPN
from torchvision.ops import misc as misc_nn_ops
from collections import OrderedDict
from torch.jit.annotations import Tuple, List, Dict, Optional

from resnest.torch import resnest101

In [ ]:
import sys
sys.path.insert(0, "../input/timmeffdetpkg")
sys.path.insert(0, "../input/omegaconf")
sys.path.insert(0, "../input/weightedboxesfusion")
sys.path.append("../input/wheatdetectiontta/external/pytorch-image-models")
sys.path.append("../input/wheatdetectiontta")
sys.path.append("../input/wheatdetectiontta/external/yacs")
import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob


from ensemble_boxes import *
import gc
from tqdm import tqdm
from effdet import get_efficientdet_config, EfficientDet, DetBenchEval, DetBenchTrain
from effdet.efficientdet import HeadNet
from skopt import gp_minimize, forest_minimize
from skopt.utils import use_named_args
from skopt.plots import plot_objective, plot_evaluations, plot_convergence, plot_regret
from skopt.space import Categorical, Integer, Real

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

# Dataset

In [ ]:
class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()

        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        
        if self.test or random.random() > 0.5:
            image, boxes = self.load_image_and_boxes(index)
        else:
            image, boxes = self.load_cutmix_image_and_boxes(index)

        # there is only one class
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    break

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        records = self.marking[self.marking['image_id'] == image_id]
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        return image, boxes

    def load_cutmix_image_and_boxes(self, index, imsize=1024):
        """ 
        This implementation of cutmix author:  https://www.kaggle.com/nvnnghia 
        Refactoring and adaptation: https://www.kaggle.com/shonenkov
        """
        w, h = imsize, imsize
        s = imsize // 2
    
        xc, yc = [int(random.uniform(imsize * 0.25, imsize * 0.75)) for _ in range(2)]  # center x, y
        indexes = [index] + [random.randint(0, self.image_ids.shape[0] - 1) for _ in range(3)]

        result_image = np.full((imsize, imsize, 3), 1, dtype=np.float32)
        result_boxes = []

        for i, index in enumerate(indexes):
            image, boxes = self.load_image_and_boxes(index)
            if i == 0:
                x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
            elif i == 1:  # top right
                x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
                x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
            elif i == 2:  # bottom left
                x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, max(xc, w), min(y2a - y1a, h)
            elif i == 3:  # bottom right
                x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)
            result_image[y1a:y2a, x1a:x2a] = image[y1b:y2b, x1b:x2b]
            padw = x1a - x1b
            padh = y1a - y1b

            boxes[:, 0] += padw
            boxes[:, 1] += padh
            boxes[:, 2] += padw
            boxes[:, 3] += padh

            result_boxes.append(boxes)

        result_boxes = np.concatenate(result_boxes, 0)
        np.clip(result_boxes[:, 0:], 0, 2 * s, out=result_boxes[:, 0:])
        result_boxes = result_boxes.astype(np.int32)
        result_boxes = result_boxes[np.where((result_boxes[:,2]-result_boxes[:,0])*(result_boxes[:,3]-result_boxes[:,1]) > 0)]
        return result_image, result_boxes

In [ ]:
class TestDatasetRetriever(Dataset):

    def __init__(self, image_ids, path, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms
        self.path = path

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{self.path}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        #image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
def get_df_folds(marking):

    df_folds = marking[['image_id']].copy()
    df_folds.loc[:, 'bbox_count'] = 1
    df_folds = df_folds.groupby('image_id').count()
    df_folds.loc[:, 'source'] = marking[['image_id', 'source']].groupby('image_id').min()['source']
    df_folds.loc[:, 'stratify_group'] = np.char.add(
        df_folds['source'].values.astype(str),
        df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
    )
    df_folds.loc[:, 'fold'] = 0

#     for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
#         df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number
    
    return df_folds

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))


def get_train_transforms(size):
    return A.Compose(
        [
            A.RandomSizedCrop(min_max_height=(800, 800), height=1024, width=1024, p=0.5),
            A.OneOf([
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                     val_shift_limit=0.2, p=0.9),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.9),
            ],p=0.9),
            A.ToGray(p=0.01),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=size, width=size, p=1),
            A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

def get_valid_transforms(size):
    return A.Compose(
        [
            A.Resize(height=size, width=size, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )


# Resnest

In [ ]:
class CrossEntropyLabelSmooth(torch.nn.Module):
    """Cross entropy loss with label smoothing regularizer.

    Reference:
    Szegedy et al. Rethinking the Inception Architecture for Computer Vision. CVPR 2016.
    Equation: y = (1 - epsilon) * y + epsilon / K.

    Args:
        num_classes (int): number of classes.
        epsilon (float): weight.
    """
    def __init__(self, num_classes, epsilon=0.1, use_gpu=True):
        super(CrossEntropyLabelSmooth, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon
        self.use_gpu = use_gpu
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, inputs, targets):
        """
        Args:
            inputs: prediction matrix (before softmax) with shape (batch_size, num_classes)
            targets: ground truth labels with shape (num_classes)
        """
        log_probs = self.logsoftmax(inputs)
        targets = torch.zeros(log_probs.size()).scatter_(1, targets.unsqueeze(1).data.cpu(), 1)
        if self.use_gpu: targets = targets.cuda()
        targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
        loss = (- targets * log_probs).mean(0).sum()
        return loss

In [ ]:
def fastrcnn_loss(class_logits, box_regression, labels, regression_targets):
    # type: (Tensor, Tensor, List[Tensor], List[Tensor]) -> Tuple[Tensor, Tensor]
    """
    Computes the loss for Faster R-CNN.
    Arguments:
        class_logits (Tensor)
        box_regression (Tensor)
        labels (list[BoxList])
        regression_targets (Tensor)
    Returns:
        classification_loss (Tensor)
        box_loss (Tensor)
    """

    labels = torch.cat(labels, dim=0)
    regression_targets = torch.cat(regression_targets, dim=0)
    labal_smooth_loss = CrossEntropyLabelSmooth(2)
    classification_loss = labal_smooth_loss(class_logits, labels)

    # get indices that correspond to the regression targets for
    # the corresponding ground truth labels, to be used with
    # advanced indexing
    sampled_pos_inds_subset = torch.nonzero(labels > 0).squeeze(1)
    labels_pos = labels[sampled_pos_inds_subset]
    N, num_classes = class_logits.shape
    box_regression = box_regression.reshape(N, -1, 4)

    box_loss = det_utils.smooth_l1_loss(
        box_regression[sampled_pos_inds_subset, labels_pos],
        regression_targets[sampled_pos_inds_subset],
        beta=1 / 9,
        size_average=False,
    )
    box_loss = box_loss / labels.numel()

    return classification_loss, box_loss

In [ ]:
def resnest_fpn_backbone(pretrained, norm_layer=misc_nn_ops.FrozenBatchNorm2d, trainable_layers=3):
    # resnet_backbone = resnet.__dict__['resnet152'](pretrained=pretrained,norm_layer=norm_layer)
    backbone = resnest101(pretrained=pretrained)
    # select layers that wont be frozen
    assert trainable_layers <= 5 and trainable_layers >= 0
    layers_to_train = ['layer4', 'layer3', 'layer2', 'layer1', 'conv1'][:trainable_layers]
    # freeze layers only if pretrained backbone is used
    for name, parameter in backbone.named_parameters():
        if all([not name.startswith(layer) for layer in layers_to_train]):
            parameter.requires_grad_(False)
    return_layers = {'layer1': '0', 'layer2': '1', 'layer3': '2', 'layer4': '3'}
    in_channels_stage2 = backbone.inplanes // 8
    in_channels_list = [
        in_channels_stage2,
        in_channels_stage2 * 2,
        in_channels_stage2 * 4,
        in_channels_stage2 * 8,
    ]
    out_channels = 256
    return BackboneWithFPN(backbone, return_layers, in_channels_list, out_channels)

In [ ]:
class WheatDetector(torch.nn.Module):
    def __init__(self, trainable_layers=3, **kwargs):
        super(WheatDetector, self).__init__()
        backbone = resnest_fpn_backbone(pretrained=False)
        self.base = FasterRCNN(backbone, num_classes = 2, **kwargs)
        self.base.roi_heads.fastrcnn_loss = fastrcnn_loss

    def forward(self, images, targets=None):
        return self.base(images, targets)

In [ ]:
from modeling import build_model
from config import cfg

def load_res_net(path, cfg):
    model = build_model(cfg)
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval();
    return model.cuda()

In [ ]:
def get_net(level):
    config = get_efficientdet_config(f'tf_efficientdet_d{level}')
    net = EfficientDet(config, pretrained_backbone=False)
    
    if level == 5:
        checkpoint = torch.load('../input/efficientdet/efficientdet_d5-ef44aea8.pth')
    elif level == 7:
        checkpoint = torch.load('../input/efficientdet/efficientdet_d7-f05bf714.pth')
        
    net.load_state_dict(checkpoint)
    config.num_classes = 1
    config.image_size = 512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    net = DetBenchTrain(net, config)
    return net

def load_net_eval(checkpoint_path, level):
    config = get_efficientdet_config(f'tf_efficientdet_d{level}')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size = 512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint["model_state_dict"])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

#net = get_net()

## Fitter

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
import math
import torch
from torch.optim.optimizer import Optimizer, required

class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = group['lr'] * math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = group['lr'] / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:            
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

In [ ]:
import warnings

warnings.filterwarnings("ignore")

class Fitter:
    
    def __init__(self, model, device, config):
        self.config = config
        self.epoch = 0

        self.base_dir = f'./{config.folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5

        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = RAdam(self.model.parameters(), lr=config.lr)
        #self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss = self.train_one_epoch(train_loader)

            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/retrain.bin')

            t = time.time()
            summary_loss = self.validation(validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-retrain.bin')

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1

    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        t = time.time()
        for step, (images, targets, image_ids) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                images = torch.stack(images)
                batch_size = images.shape[0]
                images = images.to(self.device).float()
                boxes = [target['boxes'].to(self.device).float() for target in targets]
                labels = [target['labels'].to(self.device).float() for target in targets]

                loss, _, _ = self.model(images, boxes, labels)
                summary_loss.update(loss.detach().item(), batch_size)

        return summary_loss

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        t = time.time()
        for step, (images, targets, image_ids) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            
            images = torch.stack(images)
            images = images.to(self.device).float()
            batch_size = images.shape[0]
            boxes = [target['boxes'].to(self.device).float() for target in targets]
            labels = [target['labels'].to(self.device).float() for target in targets]

            self.optimizer.zero_grad()
            
            loss, _, _ = self.model(images, boxes, labels)
            
            loss.backward()

            summary_loss.update(loss.detach().item(), batch_size)

            self.optimizer.step()

            if self.config.step_scheduler:
                self.scheduler.step()

        return summary_loss
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.model.state_dict(),
            #'optimizer_state_dict': self.optimizer.state_dict(),
            #'scheduler_state_dict': self.scheduler.state_dict(),
            #'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path, device):
        checkpoint = torch.load(path)
        self.model.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.to(device)
        #self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        #self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        #self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

# Global Setting

In [ ]:
N_FOLD = 2
USE_TTA = True
TRAIN_ROOT_PATH = '../input/global-wheat-detection/train'
TEST_ROOT_PATH = '../input/global-wheat-detection/test'
cfg.MODEL.PRETRAIN = False

In [ ]:
MODEL = {
    "effdet": [
        load_net_eval('../input/effdetd5sourcelee/best-retrain-epoch51.bin', 5), 
        load_net_eval('../input/effdetd7/retrains/best-retrain-epoch42.bin', 7)
    ],
    "resnest":[
        load_res_net("../input/resnest-source-weights-andreshuang/checkpoint-60arvalis1.bin", cfg),
        load_res_net("../input/rssnest-source-weights-cjc/best-checkpoint_PL_ethz_1.bin", cfg)
    ]
}

# Bayesian optimization using Gaussian Processes

In [ ]:
import numba
import re
import ast

from numba import jit
from typing import List, Union, Tuple


@jit(nopython=True)
def calculate_iou(gt, pr, form='pascal_voc') -> float:
    """Calculates the Intersection over Union.

    Args:
        gt: (np.ndarray[Union[int, float]]) coordinates of the ground-truth box
        pr: (np.ndarray[Union[int, float]]) coordinates of the prdected box
        form: (str) gt/pred coordinates format
            - pascal_voc: [xmin, ymin, xmax, ymax]
            - coco: [xmin, ymin, w, h]
    Returns:
        (float) Intersection over union (0.0 <= iou <= 1.0)
    """
    if form == 'coco':
        gt = gt.copy()
        pr = pr.copy()

        gt[2] = gt[0] + gt[2]
        gt[3] = gt[1] + gt[3]
        pr[2] = pr[0] + pr[2]
        pr[3] = pr[1] + pr[3]

    # Calculate overlap area
    dx = min(gt[2], pr[2]) - max(gt[0], pr[0]) + 1
    
    if dx < 0:
        return 0.0
    
    dy = min(gt[3], pr[3]) - max(gt[1], pr[1]) + 1

    if dy < 0:
        return 0.0

    overlap_area = dx * dy

    # Calculate union area
    union_area = (
            (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1) +
            (pr[2] - pr[0] + 1) * (pr[3] - pr[1] + 1) -
            overlap_area
    )

    return overlap_area / union_area


@jit(nopython=True)
def find_best_match(gts, pred, pred_idx, threshold = 0.5, form = 'pascal_voc', ious=None) -> int:
    """Returns the index of the 'best match' between the
    ground-truth boxes and the prediction. The 'best match'
    is the highest IoU. (0.0 IoUs are ignored).

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        pred: (List[Union[int, float]]) Coordinates of the predicted box
        pred_idx: (int) Index of the current predicted box
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (int) Index of the best match GT box (-1 if no match above threshold)
    """
    best_match_iou = -np.inf
    best_match_idx = -1

    for gt_idx in range(len(gts)):
        
        if gts[gt_idx][0] < 0:
            # Already matched GT-box
            continue
        
        iou = -1 if ious is None else ious[gt_idx][pred_idx]

        if iou < 0:
            iou = calculate_iou(gts[gt_idx], pred, form=form)
            
            if ious is not None:
                ious[gt_idx][pred_idx] = iou

        if iou < threshold:
            continue

        if iou > best_match_iou:
            best_match_iou = iou
            best_match_idx = gt_idx

    return best_match_idx

@jit(nopython=True)
def calculate_precision(gts, preds, threshold = 0.5, form = 'coco', ious=None) -> float:
    """Calculates precision for GT - prediction pairs at one threshold.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (float) Precision
    """
    n = len(preds)
    tp = 0
    fp = 0
    
    # for pred_idx, pred in enumerate(preds_sorted):
    for pred_idx in range(n):

        best_match_gt_idx = find_best_match(gts, preds[pred_idx], pred_idx,
                                            threshold=threshold, form=form, ious=ious)

        if best_match_gt_idx >= 0:
            # True positive: The predicted box matches a gt box with an IoU above the threshold.
            tp += 1
            # Remove the matched GT box
            gts[best_match_gt_idx] = -1

        else:
            # No match
            # False positive: indicates a predicted box had no associated gt box.
            fp += 1

    # False negative: indicates a gt box had no associated predicted box.
    fn = (gts.sum(axis=1) > 0).sum()

    return tp / (tp + fp + fn)


@jit(nopython=True)
def calculate_image_precision(gts, preds, thresholds = (0.5, ), form = 'coco') -> float:
    """Calculates image precision.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        thresholds: (float) Different thresholds
        form: (str) Format of the coordinates

    Return:
        (float) Precision
    """
    n_threshold = len(thresholds)
    image_precision = 0.0
    
    ious = np.ones((len(gts), len(preds))) * -1
    # ious = None

    for threshold in thresholds:
        precision_at_threshold = calculate_precision(gts.copy(), preds, threshold=threshold,
                                                     form=form, ious=ious)
        image_precision += precision_at_threshold / n_threshold

    return image_precision

def show_result(sample_id, preds, gt_boxes):
    sample = cv2.imread(f'{TRAIN_ROOT_PATH}/{sample_id}.jpg', cv2.IMREAD_COLOR)
    sample = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)

    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    for pred_box in preds:
        cv2.rectangle(
            sample,
            (pred_box[0], pred_box[1]),
            (pred_box[2], pred_box[3]),
            (220, 0, 0), 2
        )

    for gt_box in gt_boxes:    
        cv2.rectangle(
            sample,
            (gt_box[0], gt_box[1]),
            (gt_box[2], gt_box[3]),
            (0, 0, 220), 2
        )

    ax.set_axis_off()
    ax.imshow(sample)
    ax.set_title("RED: Predicted | BLUE - Ground-truth")
    
# Numba typed list!
iou_thresholds = numba.typed.List()

for x in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75]:
    iou_thresholds.append(x)

In [ ]:
def to_tensor(images):
    tmp = []
    for img in images:
        img = img.astype(np.float32)
        img /= 255.0
        img = torch.tensor(img, dtype=torch.float32)
        tmp.append(img.permute(2,0,1))
    return torch.stack(tmp)

def get_handout_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            #ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

class HandoutDatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, path):
        super().__init__()
        self.image_ids = image_ids
        self.marking = marking
        self.path = path

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{self.path}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        records = self.marking[self.marking['image_id'] == image_id]
        gtboxes = records[['x', 'y', 'w', 'h']].values
        gtboxes[:, 2] = gtboxes[:, 0] + gtboxes[:, 2]
        gtboxes[:, 3] = gtboxes[:, 1] + gtboxes[:, 3]
            
        return image, gtboxes, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
def make_predictions(
    models,
    images, 
    score_threshold=0.25,
):
    predictions = []
    for fold_number, net in enumerate(models):
        with torch.no_grad():
            net.eval()
            det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
            result = []
            for i in range(images.shape[0]):
                boxes = det[i].detach().cpu().numpy()[:,:4]    
                scores = det[i].detach().cpu().numpy()[:,4]
                indexes = np.where(scores > score_threshold)[0]
                boxes = boxes[indexes]
                boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                result.append({
                    'boxes': boxes[indexes],
                    'scores': scores[indexes],
                })
            predictions.append(result)
    return predictions


def run_wbf(predictions, image_index, image_size=512, iou_thr=0.5, skip_box_thr=0.4, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*1023
    return boxes, scores, labels


def TTAImage(image, index):
    image1 = image.copy()
    if index==0: 
        rotated_image = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image
    elif index==1:
        rotated_image2 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image2 = cv2.rotate(rotated_image2, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image2
    elif index==2:
        rotated_image3 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image3
    elif index == 3:
        return image1
    
    
def rotBoxes90(boxes, im_w, im_h):
    ret_boxes =[]
    for box in boxes:
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = x1-im_w//2, im_h//2 - y1, x2-im_w//2, im_h//2 - y2
        x1, y1, x2, y2 = y1, -x1, y2, -x2
        x1, y1, x2, y2 = int(x1+im_w//2), int(im_h//2 - y1), int(x2+im_w//2), int(im_h//2 - y2)
        x1a, y1a, x2a, y2a = min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)
        ret_boxes.append([x1a, y1a, x2a, y2a])
    return np.array(ret_boxes)


def run_wbf_tta(predictions, image_size=512, iou_thr=0.5, skip_box_thr=0.4, weights=None):
    boxes_1 = []
    scores_1 = []
    labels_1 = []
    for i in range(2):
        bb = (predictions['boxes'][i]/(image_size-1)).tolist()
        ss = predictions['scores'][i].tolist()
        ls = predictions['labels'][i].tolist()
        boxes_1.append(bb)
        scores_1.append(ss)
        labels_1.append(ls)

    boxes_1, scores_1, labels_1 = weighted_boxes_fusion(boxes_1, scores_1, labels_1, 
                                                        weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    boxes_2 = []
    scores_2 = []
    labels_2 = []
    for i in range(2,4):
        bb = (predictions['boxes'][i]/(image_size-1)).tolist()
        ss = predictions['scores'][i].tolist()
        ls = predictions['labels'][i].tolist()
        boxes_2.append(bb)
        scores_2.append(ss)
        labels_2.append(ls)
    
    boxes_2, scores_2, labels_2 = weighted_boxes_fusion(boxes_2, scores_2, labels_2, 
                                                        weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    boxes = [boxes_1.tolist(), boxes_2.tolist()]
    scores = [scores_1.tolist(), scores_2.tolist()]
    labels = [labels_1.tolist(), labels_2.tolist()]
    
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, 
                                                        weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    #boxes = boxes*(image_size-1)
    return boxes, scores, labels



def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)

In [ ]:
def calculate_final_score(
    all_predictions,
    iou_thr,
    skip_box_thr,
    resweight,
    edetweight,
    sigma=0.5,
):
    final_scores = []
    for predictions in all_predictions:
        
        gt_boxes = predictions['gtboxes'].copy()
        image_id = predictions['image_id']
        
        img_boxes = []
        img_scores = []
        img_labels = []
        
        for index in range(4):
            effdet_p = predictions["effdet"][index]
            boxes, scores, labels = run_wbf(effdet_p, image_size=512, image_index=0, 
                                            iou_thr=iou_thr, skip_box_thr=skip_box_thr)
            
            resnest_p = predictions["resnest"][index]
            boxes = [resnest_p["boxes"], boxes]
            scores = [resnest_p["scores"], scores]
            labels = [resnest_p["labels"], labels]
            
            boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, 
                                            weights=[resweight, edetweight], iou_thr=iou_thr, skip_box_thr=skip_box_thr)
            
            for _ in range(3-index):

                boxes = rotBoxes90(boxes, 1024, 1024)

            if index == 3:

                boxes = boxes.astype(np.int32)

            if index == 0:

                img_boxes = (boxes / 1024).tolist()
                img_scores = scores.tolist()
                img_labels = labels.tolist()

            else:

                img_boxes = img_boxes + (boxes / 1024).tolist()
                img_scores = img_scores + scores.tolist()
                img_labels = img_labels + labels.tolist()

        boxes, scores, labels = weighted_boxes_fusion([img_boxes], 
                                                      [img_scores], 
                                                      [img_labels], 
                                                        weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        boxes = (boxes*1024).astype(np.int32).clip(min=0, max=1023)
        image_precision = calculate_image_precision(gt_boxes, boxes, thresholds=iou_thresholds, form='pascal_voc')
        
        final_scores.append(image_precision)
    
    score = np.mean(final_scores)
    
    print(f"score:{score}  iou_thr:{iou_thr}  skip_box_thr:{skip_box_thr}  weights:[{resweight, edetweight}]")
    
    return score

In [ ]:
USE_OPTIMIZE = False

if USE_OPTIMIZE:
    
    marking = pd.read_csv('../input/global-wheat-detection/train.csv')

    bboxs = np.stack(marking['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
    for i, column in enumerate(['x', 'y', 'w', 'h']):
        marking[column] = bboxs[:,i]
    marking.drop(columns=['bbox'], inplace=True)

    # drop some source
    # drop_source = ["ethz_1", "arvalis_1"]
    # marking = marking[~marking['source'].isin(drop_source)]

    # drop error boxes
    marking['area'] = marking['w'] * marking['h']
    marking = marking[marking['area'] < 154200.0]
    error_boxes = [100648.0, 145360.0, 149744.0,119790.0, 106743.0]
    marking = marking[~marking['area'].isin(error_boxes)]

    # drop less boxes images
    all_train_images = marking.copy()
    all_train_images['count'] = all_train_images.apply(lambda row: 1 if np.isfinite(row.width) else 0, axis=1)
    train_images_count = all_train_images.groupby('image_id').sum().reset_index()
    indexes = train_images_count[train_images_count['count'] < 10].image_id

    marking = marking[~marking['image_id'].isin(indexes)]
    
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

    df_folds = marking[['image_id']]
    df_folds['bbox_count'] = 1
    df_folds = df_folds.groupby('image_id').count()
    df_folds['source'] = marking[['image_id', 'source']].groupby('image_id').min()['source']
    df_folds['stratify_group'] = np.char.add(
        df_folds['source'].values.astype(str),
        df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
    )

    train_index, holdout_index = next(skf.split(X=df_folds.index, y=df_folds['stratify_group']))
    df_holdout = df_folds.iloc[holdout_index].copy()
    df_folds = df_folds.iloc[train_index].copy()

    df_folds['fold'] = 0
    for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
        df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number
    
    holdout_dataset = HandoutDatasetRetriever(
        image_ids=df_holdout.index.values,
        marking=marking,
        path=TRAIN_ROOT_PATH,
    )
    
    holdout_loader = DataLoader(
        holdout_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=4,
        drop_last=False,
        collate_fn=collate_fn
    )
    
    all_predictions = []
    for images, gtboxes, image_ids in tqdm(holdout_loader, total=len(holdout_loader)):

        image = images[0]
        image = cv2.resize(image, (512, 512))
        image_res = images[0]

        predictions_tta = {"effdet":[],
                          "resnest":[],
                          "gtboxes":gtboxes[0],
                          "image_id":image_ids[0]}

        for index in range(4):
            
            roated = TTAImage(image, index)
            roated = to_tensor([roated]).cuda()
            predictions = make_predictions(MODEL["effdet"], roated)
            
            del roated
            
            predictions_tta["effdet"].append(predictions)

            roated = TTAImage(image_res, index)
            roated = to_tensor([roated]).cuda()

            predictions = {}
            predictions_1 = MODEL["resnest"][0](roated)
            predictions_2 = MODEL["resnest"][1](roated)
            predictions["boxes"] = predictions_1[0]["boxes"].tolist() + predictions_2[0]["boxes"].tolist()
            predictions["scores"] = predictions_1[0]["scores"].tolist() + predictions_2[0]["scores"].tolist()
            predictions["labels"] = predictions_1[0]["labels"].tolist() + predictions_2[0]["labels"].tolist()
            predictions_tta["resnest"].append(predictions)
            del roated
            
        all_predictions.append(predictions_tta)

In [ ]:
def log(text):
    with open('opt.log', 'a+') as logger:
        logger.write(f'{text}\n')
        
        
def optimize(space, all_predictions, n_calls=10):
    @use_named_args(space)
    def score(**params):
        log('-'*5 + 'WBF' + '-'*5)
        log(params)
        final_score = calculate_final_score(all_predictions, **params)
        log(f'final_score = {final_score}')
        log('-'*10)
        return -final_score

    return gp_minimize(func=score, dimensions=space, n_calls=n_calls)

In [ ]:
if USE_OPTIMIZE:
    space = [
        Real(0.1, 0.7, name='iou_thr'),
        Real(0.2, 0.7, name='skip_box_thr'),
        Real(1, 10, name='resweight'),
        Real(1, 10, name='edetweight'),
    ]

    opt_result = optimize(
        space, 
        all_predictions,
        n_calls=10,
    )
    
    best_final_score = -opt_result.fun
    best_iou_thr = opt_result.x[0]
    best_skip_box_thr = opt_result.x[1]
    
    
else:
    # calculated early for fast inference in submission [see version 1 with n_calls=300]
    best_final_score = 0.68687
    best_iou_thr = 0.41485
    best_skip_box_thr = 0.4159
    best_resweight = 3.62106
    best_edetweight = 6.50667
    
print('-'*13 + 'WBF' + '-'*14)
print(f'[Best Score]: {best_final_score:.4f}')
print(f'[Best Iou Thr]: {best_iou_thr:.3f}')
print(f'[Best Skip Box Thr]: {best_skip_box_thr:.3f}')
print(f'[Best Best Resnest weight]: {best_resweight:.3f}')
print(f'[Best Best Effdet weight]: {best_edetweight:.3f}')

print('-'*30)

# Make Pseudo Label

In [ ]:
def make_predictions(
    models,
    images, 
    score_threshold=0.25,
):
    predictions = []
    for fold_number, net in enumerate(models):
        with torch.no_grad():
            net.eval()
            det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
            result = []
            for i in range(images.shape[0]):
                boxes = det[i].detach().cpu().numpy()[:,:4]    
                scores = det[i].detach().cpu().numpy()[:,4]
                indexes = np.where(scores > score_threshold)[0]
                boxes = boxes[indexes]
                boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                result.append({
                    'boxes': boxes[indexes],
                    'scores': scores[indexes],
                })
            predictions.append(result)
    return predictions


def run_wbf(predictions, image_index, image_size=512, iou_thr=best_iou_thr, skip_box_thr=best_skip_box_thr, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*1023
    return boxes, scores, labels


def TTAImage(image, index):
    image1 = image.copy()
    if index==0: 
        rotated_image = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image
    elif index==1:
        rotated_image2 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image2 = cv2.rotate(rotated_image2, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image2
    elif index==2:
        rotated_image3 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image3
    elif index == 3:
        return image1
    
    
def rotBoxes90(boxes, im_w, im_h):
    ret_boxes =[]
    for box in boxes:
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = x1-im_w//2, im_h//2 - y1, x2-im_w//2, im_h//2 - y2
        x1, y1, x2, y2 = y1, -x1, y2, -x2
        x1, y1, x2, y2 = int(x1+im_w//2), int(im_h//2 - y1), int(x2+im_w//2), int(im_h//2 - y2)
        x1a, y1a, x2a, y2a = min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)
        ret_boxes.append([x1a, y1a, x2a, y2a])
    return np.array(ret_boxes)


def run_wbf_tta(predictions, image_size=512, iou_thr=best_iou_thr, skip_box_thr=best_skip_box_thr, weights=None):
    boxes_1 = []
    scores_1 = []
    labels_1 = []
    for i in range(2):
        bb = (predictions['boxes'][i]/(image_size-1)).tolist()
        ss = predictions['scores'][i].tolist()
        ls = predictions['labels'][i].tolist()
        boxes_1.append(bb)
        scores_1.append(ss)
        labels_1.append(ls)

    boxes_1, scores_1, labels_1 = weighted_boxes_fusion(boxes_1, scores_1, labels_1, 
                                                        weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    boxes_2 = []
    scores_2 = []
    labels_2 = []
    for i in range(2,4):
        bb = (predictions['boxes'][i]/(image_size-1)).tolist()
        ss = predictions['scores'][i].tolist()
        ls = predictions['labels'][i].tolist()
        boxes_2.append(bb)
        scores_2.append(ss)
        labels_2.append(ls)
    
    boxes_2, scores_2, labels_2 = weighted_boxes_fusion(boxes_2, scores_2, labels_2, 
                                                        weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    boxes = [boxes_1.tolist(), boxes_2.tolist()]
    scores = [scores_1.tolist(), scores_2.tolist()]
    labels = [labels_1.tolist(), labels_2.tolist()]
    
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, 
                                                        weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    #boxes = boxes*(image_size-1)
    return boxes, scores, labels



def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)

In [ ]:
test_dataset = TestDatasetRetriever(
    image_ids=np.array([path.split('/')[-1][:-4] for path in glob(f'{TEST_ROOT_PATH}/*.jpg')]),
    path=TEST_ROOT_PATH
)

test_data_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [ ]:
if len(os.listdir(TEST_ROOT_PATH)) > 10:
    
    marking = pd.read_csv('../input/global-wheat-detection/train.csv')

    bboxs = np.stack(marking['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
    for i, column in enumerate(['x', 'y', 'w', 'h']):
        marking[column] = bboxs[:,i]
    marking.drop(columns=['bbox'], inplace=True)

    marking_p = marking.copy()
    results = []
    for images, image_ids in test_data_loader:

        image = images[0]
        height, width, _ = image.shape
        image = cv2.resize(image, (512, 512))
        image_res = cv2.resize(image, (1024, 1024))

        predictions_tta = {
            "boxes": [],
            "scores": [],
            "labels": []
        }

        for index in range(4):
            roated = TTAImage(image, index)
            roated = to_tensor([roated]).cuda()
            predictions = make_predictions(MODEL["effdet"], roated)
            boxes, scores, labels = run_wbf(predictions, image_size=512, image_index=0)


            roated = TTAImage(image_res, index)
            roated = to_tensor([roated]).cuda()
            predictions_1 = MODEL["resnest"][0](roated)
            predictions_2 = MODEL["resnest"][1](roated)
            boxes_r = predictions_1[0]["boxes"].tolist() + predictions_2[0]["boxes"].tolist()
            scores_r = predictions_1[0]["scores"].tolist() + predictions_2[0]["scores"].tolist()
            labels_r = predictions_1[0]["labels"].tolist() + predictions_2[0]["labels"].tolist()

            boxes = [boxes_r, boxes]
            scores = [scores_r, scores]
            labels = [labels_r, labels]

            boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, 
                                                          weights=[best_resweight, best_edetweight], 
                                                          iou_thr=best_iou_thr, 
                                                          skip_box_thr=best_skip_box_thr)

            for _ in range(3-index):

                boxes = rotBoxes90(boxes, 1024, 1024)

            if index == 3:

                boxes = boxes.astype(np.int32)

            predictions_tta["boxes"].append(boxes)
            predictions_tta["scores"].append(scores)
            predictions_tta["labels"].append(labels)

        boxes, scores, labels = run_wbf_tta(predictions_tta, image_size=1024, iou_thr=best_iou_thr, skip_box_thr=best_skip_box_thr)

        boxes[:, [0, 1]] = boxes[:, [0, 1]] - boxes[:, [0, 1]]*0.01
        boxes[:, [2, 3]] = boxes[:, [2, 3]] + boxes[:, [2, 3]]*0.01

        boxes = (boxes*1024).astype(np.int32).clip(min=0, max=1023)
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]


        for bb, ss in zip(boxes, scores):
            marking_p = marking_p.append({
             'image_id':image_ids[0], 
                'width':1024, 
                'height':1024, 
                'source':"arvalis_1", 
                'x':bb[0], 'y':bb[1], 'w':bb[2], 'h':bb[3]
            }, ignore_index=True)

    marking_p = marking_p[~marking_p['image_id'].isin(marking.image_id)]

# Retrain model

In [ ]:
N_FOLD = 2
USE_OPTIMIZE = False # used for fast inference in submission
USE_TTA = True
TRAIN_ROOT_PATH = '../input/global-wheat-detection/test'
TEST_ROOT_PATH = '../input/global-wheat-detection/test'

In [ ]:
class TrainGlobalConfig:
    num_workers = 8
    batch_size = 4
    n_epochs = 5 # n_epochs = 40
    lr = 0.0002

    folder = 'retrains'

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = True  # do scheduler.step after validation stage loss

#     SchedulerClass = torch.optim.lr_scheduler.OneCycleLR
#     scheduler_params = dict(
#         max_lr=0.001,
#         epochs=n_epochs,
#         steps_per_epoch=int(len(train_dataset) / batch_size),
#         pct_start=0.1,
#         anneal_strategy='cos', 
#         final_div_factor=10**5
#     )
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=1,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )
    # --------------------

In [ ]:
def re_train(path, marker, level, folder=None):
    
    df_folds = get_df_folds(marker)
    device = torch.device('cuda:0')
    net = get_net(level)
    
    if folder:
        TrainGlobalConfig.folder = folder
    
    train_dataset = DatasetRetriever(
        image_ids=df_folds[df_folds['fold'] == 0].index.values,
        marking=marker,
        transforms=get_train_transforms(512),
        test=True,
    )

    validation_dataset = DatasetRetriever(
        image_ids=df_folds[df_folds['fold'] == 0].index.values[:len(df_folds) // 5],
        marking=marker,
        transforms=get_valid_transforms(512),
        test=True,
    )
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        sampler=RandomSampler(train_dataset),
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
        collate_fn=collate_fn,
    )
    
    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=TrainGlobalConfig.batch_size,
        num_workers=TrainGlobalConfig.num_workers,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
        collate_fn=collate_fn,
    )

    
    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
    
    fitter.load(path, device)
    
    fitter.fit(train_loader, val_loader)
    
    del fitter

In [ ]:
if len(os.listdir(TEST_ROOT_PATH)) > 10:
    
    re_train("../input/effdetd5sourcelee/best-retrain-epoch51.bin", marking_p, 5, "retrains")
    
    MODEL["effdet"][0] = load_net_eval("retrains/best-retrain.bin", 5)
 

In [ ]:
results = []
for images, image_ids in test_data_loader:

    image = images[0]
    height, width, _ = image.shape
    image = cv2.resize(image, (512, 512))
    image_res = cv2.resize(image, (1024, 1024))

    predictions_tta = {
        "boxes": [],
        "scores": [],
        "labels": []
    }

    for index in range(4):
        roated = TTAImage(image, index)
        roated = to_tensor([roated]).cuda()
        predictions = make_predictions(MODEL["effdet"], roated)
        boxes, scores, labels = run_wbf(predictions, image_size=512, image_index=0)
        
        
        roated = TTAImage(image_res, index)
        roated = to_tensor([roated]).cuda()
        predictions_1 = MODEL["resnest"][0](roated)
        predictions_2 = MODEL["resnest"][1](roated)
        boxes_r = predictions_1[0]["boxes"].tolist() + predictions_2[0]["boxes"].tolist()
        scores_r = predictions_1[0]["scores"].tolist() + predictions_2[0]["scores"].tolist()
        labels_r = predictions_1[0]["labels"].tolist() + predictions_2[0]["labels"].tolist()
        
        boxes = [boxes_r, boxes]
        scores = [scores_r, scores]
        labels = [labels_r, labels]
        
        boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, 
                                                      weights=[best_resweight, best_edetweight], 
                                                      iou_thr=best_iou_thr, 
                                                      skip_box_thr=best_skip_box_thr)
        
        for _ in range(3-index):

            boxes = rotBoxes90(boxes, 1024, 1024)

        if index == 3:

            boxes = boxes.astype(np.int32)

        predictions_tta["boxes"].append(boxes)
        predictions_tta["scores"].append(scores)
        predictions_tta["labels"].append(labels)

    boxes, scores, labels = run_wbf_tta(predictions_tta, image_size=1024, iou_thr=best_iou_thr, skip_box_thr=best_skip_box_thr)
    
    boxes[:, [0, 1]] = boxes[:, [0, 1]] - boxes[:, [0, 1]]*0.01
    boxes[:, [2, 3]] = boxes[:, [2, 3]] + boxes[:, [2, 3]]*0.01
    
    scores = scores.clip(min=0, max=1.0)
    boxes = (boxes*1024).astype(np.int32).clip(min=0, max=1023)
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
            
#     img_ = cv2.resize(images[0], (1024, 1024))
#     for bb, ss in zip(boxes, scores):
#         cv2.rectangle(img_, (bb[0], bb[1]), (bb[2]+bb[0], bb[3]+bb[1]), (220, 0, 0), 2)
#         cv2.putText(img_, '%.2f'%(ss), (bb[0], bb[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
    
#     plt.figure(figsize=(16, 8))
#     plt.imshow(img_)
#     plt.axis("off")
    
    result = {
            'image_id': image_ids[0],
            'PredictionString': format_prediction_string(boxes, scores)
        }
    
    results.append(result)

In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.to_csv('submission.csv', index=False)
test_df